In [39]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from pyspark.ml import Pipeline

In [40]:
# Iniciar a sessão do Spark
spark = SparkSession.builder.appName("RandomForest").getOrCreate()

In [42]:
# Carregar o dataset
df = spark.read.csv("pacientes_diabetes.csv", header=True, inferSchema=True)

In [43]:
# Mostrar as primeiras linhas e o schema do DataFrame
print("Amostra dos dados:")
df.show(5)
print("\nEstrutura do DataFrame:")
df.printSchema()

Amostra dos dados:
+---+----------+-----+----+----------------+--------+-----------------+-------------+
| id|data_exame|idade| imc|pressao_arterial|insulina|exercicio_semanal|nivel_glicose|
+---+----------+-----+----+----------------+--------+-----------------+-------------+
|  1|2022-06-06|   74|29.2|             118|    16.0|                3|        383.4|
|  2|2021-08-11|   87|35.5|             151|    37.1|                6|        131.1|
|  3|2022-04-07|   51|39.4|             159|    33.5|                1|        200.6|
|  4|2022-03-23|   78|35.4|             111|    47.7|                5|         86.2|
|  5|2020-11-26|   72|38.0|             127|    18.1|                6|        327.3|
+---+----------+-----+----+----------------+--------+-----------------+-------------+
only showing top 5 rows


Estrutura do DataFrame:
root
 |-- id: integer (nullable = true)
 |-- data_exame: date (nullable = true)
 |-- idade: integer (nullable = true)
 |-- imc: double (nullable = true)
 |--

In [48]:
# Prepara os dados para o modelo
feature_columns = ["idade", "imc", "pressao_arterial", "insulina", "exercicio_semanal"]
assember = VectorAssembler(inputCols=feature_columns, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Criar e treinar o modelo RandomForest
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="nivel_glicose", numTrees=100)

# Criar o pipeline
pipeline = Pipeline(stages=[assember, scaler, rf])

# Dividir o dataset em treino e teste
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Ajustar o Pipeline ao conjunto de treino
model = pipeline.fit(train_data)

# Fazer previsões no conjunto de teste
predictions = model.transform(test_data)

# Avaliar o modelo
'''
O Erro Quadrático Médio (RMSE - Root Mean Squared Error, em inglês) é uma métrica que mede a média dos quadrados dos erros, ou seja, a diferença entre o valor real e o valor previsto ao quadrado.
    - O RMSE é expresso na mesma unidade da variável alvo, o que facilita a interpretação.
    - Quanto menor o RMSE, melhor o modelo.
    - Um RMSE de 0 indica que o modelo prevê os valores com perfeição.
'''
evaluator = RegressionEvaluator(labelCol="nivel_glicose", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"\nErro Quadrático Médio (RMSE): {rmse}")


Erro Quadrático Médio (RMSE): 94.96183762006302


In [59]:
# Função para prever o nível de glicose de um novo paciente
def prever_nivel_glicose(idade, imc, pressao_arterial, insulina, exercicio_semanal):
    print("\nPrevisão para um novo paciente:")
    print(f"Idade: {idade}")
    print(f"IMC: {imc}")
    print(f"Pressão Arterial: {pressao_arterial}")
    print(f"Insulina: {insulina}")
    print(f"Exercício Semanal: {exercicio_semanal}")
    
    data = [(idade, imc, pressao_arterial, insulina, exercicio_semanal)]
    schema=StructType([
        StructField("idade", IntegerType(), True),
        StructField("imc", FloatType(), True),
        StructField("pressao_arterial", IntegerType(), True),
        StructField("insulina", IntegerType(), True),
        StructField("exercicio_semanal", IntegerType(), True)
    ])
    novo_df_com_paciente = spark.createDataFrame(data, schema)
    prediction = model.transform(novo_df_com_paciente).head()

    # print(novo_df_com_paciente.show())
    # print(prediction)
    # print(prediction["prediction"])

    return prediction["prediction"]

In [60]:
novo_paciente1 = (50, 30.0, 80, 100, 2)
nivel_glicose1 = prever_nivel_glicose(*novo_paciente1)
print(f"\nO nível de glicose previsto para o novo paciente 1 é: {nivel_glicose1:.2f}")

novo_paciente2 = (30, 25.0, 70, 50, 3)
nivel_glicose2 = prever_nivel_glicose(*novo_paciente2)
print(f"\nO nível de glicose previsto para o novo paciente 2 é: {nivel_glicose2:.2f}")

novo_paciente3 = (30, 25.0, 120, 10, 3)
nivel_glicose3 = prever_nivel_glicose(*novo_paciente3)
print(f"\nO nível de glicose previsto para o novo paciente 3 é: {nivel_glicose3:.2f}")


Previsão para um novo paciente:
Idade: 50
IMC: 30.0
Pressão Arterial: 80
Insulina: 100
Exercício Semanal: 2

O nível de glicose previsto para o novo paciente 1 é: 240.01

Previsão para um novo paciente:
Idade: 30
IMC: 25.0
Pressão Arterial: 70
Insulina: 50
Exercício Semanal: 3

O nível de glicose previsto para o novo paciente 2 é: 236.92

Previsão para um novo paciente:
Idade: 30
IMC: 25.0
Pressão Arterial: 120
Insulina: 10
Exercício Semanal: 3

O nível de glicose previsto para o novo paciente 3 é: 235.52


In [38]:

# Encerrar a sessão do Spark
spark.stop()